# Animal Shelter Data
---

The datasets explored for this project cover Austin and Dallas Texas.  Working with datasets from different municipalities should highlight the importance of particular data as well as the significance of omitting some data.  

Starting from this vantage point the data is going to be cleaned in a way to collect the most meaningful data that will improve the issues facing animal shelters.

Problem Statement...

---
## Data Cleaning

In [1]:
# imports
import pandas as pd
import numpy as np

### Austin Shelter Intakes

In [2]:
# reading in shelter intakes
intakes =  pd.read_csv('../data/austin_animal_center_intakes_20241017.csv')
intakes.head()

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
0,A786884,*Brock,01/03/2019 04:19:00 PM,January 2019,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor
1,A706918,Belle,07/05/2015 12:59:00 PM,July 2015,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver
2,A724273,Runster,04/14/2016 06:43:00 PM,April 2016,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White
3,A665644,NaN,10/21/2013 07:59:00 AM,October 2013,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico
4,A857105,Johnny Ringo,05/12/2022 12:23:00 AM,May 2022,4404 Sarasota Drive in Austin (TX),Public Assist,Normal,Cat,Neutered Male,2 years,Domestic Shorthair,Orange Tabby


In [3]:
intakes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168040 entries, 0 to 168039
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Animal ID         168040 non-null  object
 1   Name              119647 non-null  object
 2   DateTime          168040 non-null  object
 3   MonthYear         168040 non-null  object
 4   Found Location    168040 non-null  object
 5   Intake Type       168040 non-null  object
 6   Intake Condition  168040 non-null  object
 7   Animal Type       168040 non-null  object
 8   Sex upon Intake   168038 non-null  object
 9   Age upon Intake   168039 non-null  object
 10  Breed             168040 non-null  object
 11  Color             168040 non-null  object
dtypes: object(12)
memory usage: 15.4+ MB


*The Name column has a lot of null values and we're assuming a pet's name won't affect their chances of adoption, so going to drop this column. Also dropping MonthYear because that information is also in the DateTime column.*

In [4]:
# dropping inconsequential columns
intakes.drop(columns=['Name', 'MonthYear'], inplace=True)

In [5]:
# renaming columns to be intake specific and snake case
columns = {
    'Animal ID': 'animal_id',
    'DateTime': 'intake_time',
    'Found Location': 'found_location',
    'Intake Type': 'intake_type',
    'Intake Condition': 'intake_condition',
    'Animal Type': 'animal_type',
    'Sex upon Intake': 'intake_gender',
    'Age upon Intake': 'intake_age',
    'Breed': 'intake_breed',
    'Color': 'intake_color'    
}

intakes = intakes.rename(columns=columns)

In [6]:
# converting intake_time column to datetime format
intakes['intake_time'] = pd.to_datetime(intakes['intake_time'], format='%m/%d/%Y %I:%M:%S %p')

In [7]:
intakes.nunique()

animal_id           151007
intake_time         115852
found_location       68164
intake_type              6
intake_condition        20
animal_type              5
intake_gender            5
intake_age              55
intake_breed          2969
intake_color           651
dtype: int64

*There are many animals that have more than one stay at a shelter. In order to have accurate merging between intake and outcomes we are separating any duplicate animals.*

In [8]:
# sort intakes by most recent intakes first
intakes.sort_values(by=['intake_time', 'animal_id'], inplace=True, ascending = False)

In [9]:
# creating a dataframe for repeat animals - note the keeping the first
repeat_intakes = intakes[intakes.duplicated(subset=['animal_id'], keep=False)].sort_values(by=['animal_id'])

# drop duplicate observations
intakes.drop_duplicates(subset='animal_id', inplace = True, keep=False)

### Austin Shelter Outcomes

In [10]:
# reading in shelter outcomes
outcomes = pd.read_csv('../data/austin_animal_center_outcomes_20241017.csv')
outcomes.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A882831,*Hamilton,07/01/2023 06:12:00 PM,Jul 2023,03/25/2023,Adoption,NaN,Cat,Neutered Male,3 months,Domestic Shorthair Mix,Black/White
1,A794011,Chunk,05/08/2019 06:20:00 PM,May 2019,05/02/2017,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
2,A776359,Gizmo,07/18/2018 04:02:00 PM,Jul 2018,07/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown
3,A821648,NaN,08/16/2020 11:38:00 AM,Aug 2020,08/16/2019,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray
4,A720371,Moose,02/13/2016 05:59:00 PM,Feb 2016,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff


In [11]:
outcomes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167942 entries, 0 to 167941
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Animal ID         167942 non-null  object
 1   Name              119733 non-null  object
 2   DateTime          167942 non-null  object
 3   MonthYear         167942 non-null  object
 4   Date of Birth     167942 non-null  object
 5   Outcome Type      167896 non-null  object
 6   Outcome Subtype   77144 non-null   object
 7   Animal Type       167942 non-null  object
 8   Sex upon Outcome  167940 non-null  object
 9   Age upon Outcome  167926 non-null  object
 10  Breed             167942 non-null  object
 11  Color             167942 non-null  object
dtypes: object(12)
memory usage: 15.4+ MB


*Dropping Name and MonthYear columns for outcomes data as well as Outcome Subtype. This column has even more null values than Name and we'll be focusing on the primary Outcome Type only.*

In [12]:
# dropping inconsequential columns
outcomes.drop(columns=['Name', 'MonthYear', 'Outcome Subtype'], inplace=True)

In [13]:
# renaming columns to be outcome specific and snake case
outcome_columns = {
    'Animal ID': 'animal_id',
    'DateTime': 'outcome_time',
    'Date of Birth': 'date_of_birth',
    'Outcome Type': 'outcome_type',
    'Animal Type': 'outcome_animal_type',
    'Sex upon Outcome': 'outcome_gender',
    'Age upon Outcome': 'outcome_age',
    'Breed': 'outcome_breed',
    'Color': 'outcome_color'    
}

outcomes = outcomes.rename(columns=outcome_columns)

In [14]:
# converting outcome_time and date of birth columns to datetime format
outcomes['outcome_time'] = pd.to_datetime(outcomes['outcome_time'], format='%m/%d/%Y %I:%M:%S %p')
outcomes['date_of_birth'] = pd.to_datetime(outcomes['date_of_birth'], format='%m/%d/%Y')

In [15]:
outcomes.nunique()

animal_id              150912
outcome_time           140118
date_of_birth            8501
outcome_type               11
outcome_animal_type         5
outcome_gender              5
outcome_age                55
outcome_breed            2969
outcome_color             653
dtype: int64

*Separating duplicate animal_id's for outcomes as well.*

In [16]:
# sort intakes by most recent intakes first
outcomes.sort_values(by=['outcome_time', 'animal_id'], inplace=True, ascending = False)

In [17]:
# creating a dataframe for repeat animals - note the keeping the first
repeat_outcomes = outcomes[outcomes.duplicated(subset=['animal_id'], keep=False)].sort_values(by=['animal_id'])

# drop duplicate observations
outcomes.drop_duplicates(subset='animal_id', inplace = True, keep=False)

### Merge Austin DataFrames

The animal_ids that show up more than once in the intakes and outcomes have been separated.  This will allow the merge between animals in the system once to be handled cleanly, particularly the maajority being single stay animals.

There is a longer process to merge the repeat animals.  In order to achieve the best merge a few temporary columns will be created to sequentially order observations.  This will allow for the creation of a unique animal_stay number, combination of animal id and the instance in the shelter.

In [18]:
# adding temporary column to indicate intakes and outcomes
repeat_intakes['intakes'] = 'intakes'
repeat_outcomes['intakes'] = 'outcomes'

In [19]:
# checking for missing values and shape
repeat_intakes.isna().sum().sum(), repeat_intakes.shape

(0, (30119, 11))

In [20]:
# checking for missing values and shape
repeat_outcomes.isna().sum().sum(), repeat_outcomes.shape

(11, (30117, 10))

In [21]:
# finding na values
repeat_outcomes.isna().sum()

animal_id               0
outcome_time            0
date_of_birth           0
outcome_type           11
outcome_animal_type     0
outcome_gender          0
outcome_age             0
outcome_breed           0
outcome_color           0
intakes                 0
dtype: int64

In [22]:
# dropping missing values in outcome_type
repeat_outcomes.dropna(subset=['outcome_type'], inplace=True)

In [23]:
# The two dataframes are concat together.  This allows to sort all observations sequentially.
# A column is added to create a date column.
df = pd.concat([repeat_intakes, repeat_outcomes])
df['sequential_date'] = df['outcome_time']
df['sequential_date'] = df['sequential_date'].fillna(df['intake_time'])

df.sort_values(by=['animal_id','sequential_date'], inplace=True, ascending=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60225 entries, 167431 to 113926
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   animal_id            60225 non-null  object        
 1   intake_time          30119 non-null  datetime64[ns]
 2   found_location       30119 non-null  object        
 3   intake_type          30119 non-null  object        
 4   intake_condition     30119 non-null  object        
 5   animal_type          30119 non-null  object        
 6   intake_gender        30119 non-null  object        
 7   intake_age           30119 non-null  object        
 8   intake_breed         30119 non-null  object        
 9   intake_color         30119 non-null  object        
 10  intakes              60225 non-null  object        
 11  outcome_time         30106 non-null  datetime64[ns]
 12  date_of_birth        30106 non-null  datetime64[ns]
 13  outcome_type         30106 non

In [24]:
# split intakes and outcomes for sequential identifiers
df1 = df[df['intakes'] == 'intakes'].copy()
df2 = df[df['intakes'] == 'outcomes'].copy()
df1.shape, df2.shape

((30119, 20), (30106, 20))

In [25]:
# dropping empty columns manually to control which are dropped
df1_cols_drop = ['outcome_time', 'date_of_birth', 'outcome_type',
       'outcome_animal_type', 'outcome_gender', 'outcome_age', 'outcome_breed',
       'outcome_color']

df2_cols_drop = ['intake_time', 'found_location', 'intake_type',
       'intake_condition', 'animal_type', 'intake_gender', 'intake_age',
       'intake_breed', 'intake_color']

df1.drop(columns=df1_cols_drop, inplace=True)
df2.drop(columns=df2_cols_drop, inplace=True)
df1.shape, df2.shape

((30119, 12), (30106, 11))

In [26]:
# creating a unique identifier, animal_stay, to tie with outcomes
df1.sort_values(by=['animal_id', 'sequential_date'], inplace=True)
df1['stay'] = df1.groupby('animal_id').cumcount() +1 
df1['stay'] = df1['stay'].astype('str')
df1['animal_stay'] = df1['animal_id'] + '-' + df1['stay']
df1.shape

(30119, 14)

In [27]:
# creating a unique identifier, animal_stay, to tie with intakes
df2.sort_values(by=['animal_id', 'sequential_date'], inplace=True)
df2['stay'] = df2.groupby('animal_id').cumcount() +1 
df2['stay'] = df2['stay'].astype('str')
df2['animal_stay'] = df2['animal_id'] + '-' + df2['stay']
df2.shape

(30106, 13)

In [28]:
# merging the two dataframes
repeats = pd.merge(left=df2, right=df1, how='inner', on='animal_stay')
repeats.shape

(29902, 26)

In [29]:
repeats.columns

Index(['animal_id_x', 'intakes_x', 'outcome_time', 'date_of_birth',
       'outcome_type', 'outcome_animal_type', 'outcome_gender', 'outcome_age',
       'outcome_breed', 'outcome_color', 'sequential_date_x', 'stay_x',
       'animal_stay', 'animal_id_y', 'intake_time', 'found_location',
       'intake_type', 'intake_condition', 'animal_type', 'intake_gender',
       'intake_age', 'intake_breed', 'intake_color', 'intakes_y',
       'sequential_date_y', 'stay_y'],
      dtype='object')

In [30]:
# cleaning up column namesf and adding repeats column indicator
repeats.drop(columns=['intakes_x', 'sequential_date_x', 'stay_x', 'animal_id_y', 'intakes_y', 'sequential_date_y'], inplace=True)
repeats = repeats.rename(columns={'animal_id_x': 'animal_id', 'stay_y': 'stay'})
repeats['stay'] = repeats['stay'].astype(int)
repeats['repeat'] = 1

In [31]:
repeats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29902 entries, 0 to 29901
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   animal_id            29902 non-null  object        
 1   outcome_time         29902 non-null  datetime64[ns]
 2   date_of_birth        29902 non-null  datetime64[ns]
 3   outcome_type         29902 non-null  object        
 4   outcome_animal_type  29902 non-null  object        
 5   outcome_gender       29902 non-null  object        
 6   outcome_age          29902 non-null  object        
 7   outcome_breed        29902 non-null  object        
 8   outcome_color        29902 non-null  object        
 9   animal_stay          29902 non-null  object        
 10  intake_time          29902 non-null  datetime64[ns]
 11  found_location       29902 non-null  object        
 12  intake_type          29902 non-null  object        
 13  intake_condition     29902 non-

In [32]:
# meging intakes and outcomes dataframes
intakes_outcomes = pd.merge(left=outcomes, right=intakes, how='inner', on='animal_id')
intakes_outcomes['stay'] = 1
intakes_outcomes['repeat'] = 0
intakes_outcomes['animal_stay'] = intakes_outcomes['animal_id'] + '-' + intakes_outcomes['stay'].astype('str')
intakes_outcomes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136920 entries, 0 to 136919
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   animal_id            136920 non-null  object        
 1   outcome_time         136920 non-null  datetime64[ns]
 2   date_of_birth        136920 non-null  datetime64[ns]
 3   outcome_type         136887 non-null  object        
 4   outcome_animal_type  136920 non-null  object        
 5   outcome_gender       136918 non-null  object        
 6   outcome_age          136904 non-null  object        
 7   outcome_breed        136920 non-null  object        
 8   outcome_color        136920 non-null  object        
 9   intake_time          136920 non-null  datetime64[ns]
 10  found_location       136920 non-null  object        
 11  intake_type          136920 non-null  object        
 12  intake_condition     136920 non-null  object        
 13  animal_type   

In [33]:
# merging the two dataframes
austin = pd.concat([intakes_outcomes, repeats])
austin.shape

(166822, 21)

In [34]:
austin.isna().sum()

animal_id               0
outcome_time            0
date_of_birth           0
outcome_type           33
outcome_animal_type     0
outcome_gender          2
outcome_age            16
outcome_breed           0
outcome_color           0
intake_time             0
found_location          0
intake_type             0
intake_condition        0
animal_type             0
intake_gender           2
intake_age              1
intake_breed            0
intake_color            0
stay                    0
repeat                  0
animal_stay             0
dtype: int64

In [35]:
print(austin.isna().sum().sum())
austin.dropna(inplace=True)
print(austin.isna().sum().sum())

54
0


In [36]:
# adding column for stay duration
austin['stay_duration'] = austin['outcome_time'] - austin['intake_time']

# convert stay_duration to number of days
austin['stay_duration'] = austin['stay_duration'].dt.days.astype(int)

# check for negative stay_duration
austin[austin['stay_duration'] < 0]['stay_duration'].value_counts()

stay_duration
-1       633
-128       1
-8         1
-1105      1
-69        1
-1445      1
-14        1
-385       1
-1128      1
-468       1
-982       1
-3         1
-1256      1
-2006      1
-27        1
-51        1
-65        1
-107       1
-104       1
-42        1
-4         1
-5         1
-26        1
-40        1
-118       1
-517       1
-412       1
-1220      1
-190       1
-114       1
-9         1
-23        1
-524       1
-17        1
-934       1
-383       1
-54        1
-309       1
-22        1
-219       1
-62        1
-2746      1
-30        1
-21        1
-999       1
-564       1
Name: count, dtype: int64

There are 633 stays of -1 days. We are making the assumption this is because of reporting errors between including or excluding times and or incorrectly recording AM and PM.  We are going to assume they are all zero.  

There are another 45 individual observations with negative values.  These seem to be typos.  We will drop these as no way of confirming actual values and small number of observations.

In [37]:
# change -1 day stay durations to 0
austin['stay_duration'] = austin['stay_duration'].map(lambda x: 0 if x == -1 else x)

# drop observations with a negative stay duration
print(austin.shape)
austin = austin[austin['stay_duration'] >= 0]
print(austin.shape)

(166773, 22)
(166728, 22)


In [38]:
# check for durations over 365
austin[austin['stay_duration'] <= 365]['stay_duration'].value_counts()

stay_duration
0      31996
4      15383
1      10167
3       9804
5       9354
       ...  
337        2
359        1
350        1
324        1
335        1
Name: count, Length: 366, dtype: int64

In [39]:
len(austin[austin['stay_duration'] > 365])

537

Austin has a threshhold for animals length of stay.  Once an animal has been in the shelter for a year it is placed on an urgent placement list.  It appears on the website there is currently one animal above 365 days, 2 approaching 365 days, and another 6 animals over 180 days (10/28).

In this context, we will remove stay durations about 365 as they appear to be definitionally exceptions and or clerical mistakes, 537 in total.

https://www.austintexas.gov/page/urgent-placement#:~:text=Terminology&text=Currently%2C%20any%20dog%20that%20has,automatically%20added%20to%20this%20list.&text=When%20we%20reach%20a%20level,only%20provide%20so%20much%20relief 


In [40]:
print(austin.shape)
austin = austin[austin['stay_duration'] <= 365]
print(austin.shape)

(166728, 22)
(166191, 22)


In [41]:
# compare intake/outcome animal_type, gender, breed, and color
print(f'Number of animal type changes: {austin[austin['animal_type'] != austin['outcome_animal_type']].shape[0]}')
print(f'Number of neuters/spays: {austin[austin['intake_gender'] != austin['outcome_gender']].shape[0]}')
print(f'Number of breed changes: {austin[austin['intake_breed'] != austin['outcome_breed']].shape[0]}')
print(f'Number of color changes: {austin[austin['intake_color'] != austin['outcome_color']].shape[0]}')

Number of animal type changes: 0
Number of neuters/spays: 67752
Number of breed changes: 0
Number of color changes: 0


*No changes in animal type, breed, or color from intake to outcome, dropping the duplicate column. Also making column showing if an animal is neutered or spayed while in the shelter.*

In [42]:
# dropping duplicate columns
austin.drop(columns=['outcome_animal_type', 'outcome_breed', 'outcome_color'], inplace=True)

# renaming columns without intake specifier
austin.rename(columns={'intake_breed': 'breed', 'intake_color': 'color'}, inplace=True)

In [43]:
# adding column for animals spayed-neutered while in shelters
austin['spay_neuter'] = (austin['intake_gender'] != austin['outcome_gender']).astype(int)

In [44]:
# function for converting age columns to age in months
def convert_age(age): 
    '''
    Convert an age in string to age in months.
   
    Argument:
    age(str): The animal age in str format, eg. '7 years'.
   
    Return:
    float: The age converted to months. Return 0 if the unit is not list int the fucntion.
    '''
    value, unit = age.split()
    value = abs(int(value)) # assume the nagetive age is typo 
    
    if 'year' in unit:
        return value * 12
    elif 'month' in unit:
        return value
    elif 'week' in unit:
        return round(float(value * 0.23), 2)
    elif 'day' in unit:
        return round(float(value * 0.033), 2)
    else:
        return 0

In [45]:
austin['intake_age'] = austin['intake_age'].map(convert_age)
austin['outcome_age'] = austin['outcome_age'].map(convert_age)

In [46]:
# checking animal types
austin['animal_type'].value_counts(normalize=True)

animal_type
Dog          0.545625
Cat          0.397049
Other        0.052115
Bird         0.005054
Livestock    0.000156
Name: proportion, dtype: float64

In [47]:
# see breeds under Other animal type
austin[austin['animal_type'] == 'Other']['breed'].unique()

array(['Guinea Pig', 'Bat', 'Raccoon', 'Squirrel', 'Bat/Mex Free-Tail',
       'Opossum', 'Fox', 'Lizard/Gecko', 'Tortoise', 'Rabbit Sh', 'Deer',
       'Skunk', 'Rat', 'Jersey Wooly', 'Ringtail', 'Snake', 'Lop-French',
       'Ferret', 'Cottontail', 'Angora-English Mix',
       'Turtle/Redeared Slider', 'Lizard', 'Rabbit Sh Mix', 'Chinchilla',
       'Coyote', 'Hamster', 'Lizard/Bearded Dragon', 'Hedgehog',
       'Flemish Giant', 'Californian', 'Lop-Holland', 'Rex Mix',
       'Lop-Holland Mix', 'Californian Mix', 'Lionhead', 'Lop-Mini',
       'Himalayan', 'Turtle', 'Lionhead Mix', 'Dutch Mix',
       'Rabbit Sh/Dwarf Hotot', 'Gerbil', 'Rabbit Lh', 'Lop-English Mix',
       'Angora-English', 'Snake/Python', 'New Zealand Wht/Lop-Holland',
       'Lop-Amer Fuzzy', 'Rex', 'English Spot Mix', 'Hotot',
       'New Zealand Wht', 'Harlequin Mix', 'Armadillo', 'Rex-Mini',
       'Dutch', 'English Spot', 'Cold Water', 'Chinchilla-Stnd',
       'Lop-Mini/Hotot', 'Mouse', 'Dwarf Hotot', 'Nethe

*There are very fiew observations labeled as Bird or Livestock. Animals labeled as Other contains some household pets but also a lot of wildlife that doesn't pertain to our problem statement. Dropping everything that isn't a cat or dog.*

In [48]:
# dropping obsevations that aren't cats or dogs
print(austin.shape)
austin = austin[austin['animal_type'] != 'Other']
austin = austin[austin['animal_type'] != 'Bird']
austin = austin[austin['animal_type'] != 'Livestock']
austin.shape

(166191, 20)


(156664, 20)

In [49]:
# checking remaining null values
austin.isnull().sum()

animal_id           0
outcome_time        0
date_of_birth       0
outcome_type        0
outcome_gender      0
outcome_age         0
intake_time         0
found_location      0
intake_type         0
intake_condition    0
animal_type         0
intake_gender       0
intake_age          0
breed               0
color               0
stay                0
repeat              0
animal_stay         0
stay_duration       0
spay_neuter         0
dtype: int64

In [50]:
austin.head()

,animal_id,outcome_time,date_of_birth,outcome_type,outcome_gender,outcome_age,intake_time,found_location,intake_type,intake_condition,animal_type,intake_gender,intake_age,breed,color,stay,repeat,animal_stay,stay_duration,spay_neuter
1,A912799,2024-10-17 13:07:00,2024-07-21,Adoption,Spayed Female,2.0,2024-09-05 14:57:00,7201 Levander Loop in Austin (TX),Abandoned,Normal,Cat,Intact Female,1.0,Domestic Shorthair,Brown Tabby,1,0,A912799-1,41,1
3,A912055,2024-10-17 12:25:00,2023-10-25,Adoption,Neutered Male,11.0,2024-08-25 08:20:00,1800 Fairlawn Lane in Austin (TX),Stray,Injured,Cat,Intact Male,10.0,Domestic Shorthair,Brown Tabby/White,1,0,A912055-1,53,1
4,A915002,2024-10-17 12:21:00,2023-10-10,Return to Owner,Intact Male,12.0,2024-10-10 12:10:00,Austin (TX),Public Assist,Normal,Dog,Intact Male,12.0,German Shepherd Mix,Tan,1,0,A915002-1,7,0
5,A912548,2024-10-17 11:45:00,2021-09-02,Adoption,Neutered Male,36.0,2024-09-02 22:31:00,6900 Bryn Mawr in Austin (TX),Stray,Normal,Dog,Intact Male,36.0,Siberian Husky Mix,Black/White,1,0,A912548-1,44,1
9,A915279,2024-10-17 00:00:00,2022-10-14,Transfer,Intact Female,24.0,2024-10-14 11:47:00,14514 Highsmith Street in Austin (TX),Stray,Normal,Cat,Intact Female,24.0,Domestic Shorthair,Black,1,0,A915279-1,2,0


In [51]:
# saving combined data to use in other notebooks
austin.to_csv('../data/austin-data.csv', index=False)

---
## Dallas Shelters

In [32]:
# reading in data for 2014-2015
dallas_2014 = pd.read_csv('../data/Dallas_Animal_Shelter_Data_Fiscal_Year_2014_-_2015_20241028.csv')

C:\Users\_Cramer_\AppData\Local\Temp\ipykernel_15248\3009554682.py:2: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  dallas_2014 = pd.read_csv('../data/Dallas_Animal_Shelter_Data_Fiscal_Year_2014_-_2015_20241028.csv')


In [33]:
pd.set_option('display.max_columns', 35)
dallas_2014.head()

,Animal Id,Animal Type,Animal Breed,Kennel Number,Kennel Status,Tag Type,Activity Number,Activity Sequence,Source Id,Census Tract,Council District,Intake Type,Intake Subtype,Intake Total,Reason,Staff Id,Intake Date,Intake Time,Due Out,Intake Condition,Hold Request,Outcome Type,Outcome Date,Outcome Time,Receipt Number,Impound Number,Service Request Number,Outcome Condition,Chip Status,Animal Origin,Additional Information,Month,Year
0,A0000575,CAT,DOMESTIC SH,AC 035,UNAVAILABLE,NaN,NaN,1,P0671044,W,W,STRAY,CONFINED,1,NaN,SN,10/02/2014 12:00:00 AM,12/31/1899 11:56:00 AM,10/06/2014 12:00:00 AM,TREATABLE REHABILITABLE NON-CONTAGIOUS,ADOPTION,ADOPTION,10/12/2014 12:00:00 AM,12/31/1899 03:25:00 PM,R14-372380,K14-297573,NaN,TREATABLE REHABILITABLE NON-CONTAGIOUS,SCAN NO CHIP,OVER THE COUNTER,ADOPTED,OCT.2014,FY2015
1,A0008962,DOG,LABRADOR RETR,LFD 088,LAB,NaN,NaN,1,P0053980,75218,18,CONFISCATED,KEEP SAFE,1,NaN,MB,09/24/2015 12:00:00 AM,12/31/1899 03:50:00 PM,10/03/2015 12:00:00 AM,TREATABLE REHABILITABLE NON-CONTAGIOUS,NaN,EUTHANIZED,10/04/2015 12:00:00 AM,12/31/1899 12:22:00 PM,NaN,K15-328347,442631,TREATABLE MANAGEABLE NON-CONTAGIOUS,SCAN NO CHIP,FIELD,NaN,SEP.2015,FY2015
2,A0121376,DOG,GERM SHEPHERD,LFD 042,LAB,NaN,NaN,1,P0661191,39A,9A,STRAY,CONFINED,1,NaN,MB,05/01/2015 12:00:00 AM,12/31/1899 12:09:00 PM,05/02/2015 12:00:00 AM,TREATABLE MANAGEABLE NON-CONTAGIOUS,NaN,EUTHANIZED,05/03/2015 12:00:00 AM,12/31/1899 11:53:00 AM,NaN,K15-314218,NaN,TREATABLE MANAGEABLE NON-CONTAGIOUS,SCAN CHIP,FIELD,NaN,MAY.2015,FY2015
3,A0129114,CAT,DOMESTIC SH,PSCAT 11,UNAVAILABLE,NaN,NaN,1,P0055049,75243,43,OWNER SURRENDER,GENERAL,1,ALLERGIC,CBM/JS,09/19/2015 12:00:00 AM,12/31/1899 04:46:00 PM,09/22/2015 12:00:00 AM,TREATABLE REHABILITABLE NON-CONTAGIOUS,EVERYDAY ADOPTION CENTER,ADOPTION,10/26/2015 12:00:00 AM,12/31/1899 02:09:00 PM,R15-425259,K15-327996,NaN,TREATABLE REHABILITABLE NON-CONTAGIOUS,SCAN CHIP,OVER THE COUNTER,VOMIT 5X 9/20,SEP.2015,FY2015
4,A0157434,DOG,ROTTWEILER,FREEZER,UNAVAILABLE,NaN,NaN,1,P0093154,38G,8G,OWNER SURRENDER,- DEAD ON ARRIVAL,1,NaN,DD,12/03/2014 12:00:00 AM,12/31/1899 08:06:00 PM,12/03/2014 12:00:00 AM,UNHEALTHY UNTREATABLE NON-CONTAGIOUS,NaN,DEAD ON ARRIVAL,12/04/2014 12:00:00 AM,12/31/1899 12:00:00 PM,NaN,K14-302641,NaN,UNHEALTHY UNTREATABLE NON-CONTAGIOUS,SCAN NO CHIP,FIELD,NaN,DEC.2014,FY2015


*The time of intake and outcome are split between two columns, one for the data and one for the time of day. Since we're only evaluating how long an animal is in a shelter with how many days we'll only convert the date to datetime and sort by that. Also converting column names to snake case.*

In [34]:
# function to convert column names
def to_snake_case(columns):
    '''
    Convert dataframe column names to snake case

    Keyword arguments:
    columns -- original column names of dataframe

    Returns a dictionary to pass into pandas rename function
    '''
    return {column: column.lower().replace(' ', '_') for column in columns}

In [35]:
dallas_2014.rename(columns = to_snake_case(dallas_2014.columns), inplace = True)

In [36]:
# converting intake_date and outcome_date to datetime
dallas_2014['intake_date'] = pd.to_datetime(dallas_2014['intake_date'], format='%m/%d/%Y %I:%M:%S %p')
dallas_2014['outcome_date'] = pd.to_datetime(dallas_2014['outcome_date'], format='%m/%d/%Y %I:%M:%S %p')

# sorting by outcome_date with most recent first
dallas_2014.sort_values(by = 'outcome_date', ascending = False, inplace = True)

*Will read in the datasets from other years and combine before looking at nulls, inconsequential columns, and duplicates similar to the Austin data.*

In [37]:
# function to read in data from remainging years
def read_in_dallas_data(year):
    '''
    Function to read in csv data for animal shelter data by year

    Keyword arguments:
    year -- which year to bring in data from

    Returns dataframe with column names converted to snake case,
    intake and outcome dates converted to datetime,
    and sorted by outcome date descending.
    '''
    year = str(year)
    # read in data
    if year != '2023':
        df = pd.read_csv('../data/Dallas_Animal_Shelter_Data_Fiscal_Year_'+year+'_-_'+str(int(year)+1)+'_20241028.csv', low_memory=False)
    else:
        df = pd.read_csv('../data/Dallas_Animal_Shelter_Data_Fiscal_Year_'+year+'_-_'+str(int(year)+2)+'_20241028.csv', low_memory=False)
    # convert column names
    df.rename(columns=to_snake_case(df.columns), inplace = True)
    # convert intake and outcome dates to datetime
    df['intake_date'] = pd.to_datetime(df['intake_date'], format='mixed')
    df['outcome_date'] = pd.to_datetime(df['outcome_date'], format='mixed')
    # sort by outcome_date
    df.sort_values(by = 'outcome_date', ascending = False, inplace = True)
    return df

In [38]:
dallas_2015 = read_in_dallas_data(2015)
dallas_2016 = read_in_dallas_data(2016)
dallas_2017 = read_in_dallas_data(2017)
dallas_2018 = read_in_dallas_data(2018)
dallas_2019 = read_in_dallas_data(2019)
dallas_2020 = read_in_dallas_data(2020)
dallas_2021 = read_in_dallas_data(2021)
dallas_2022 = read_in_dallas_data(2022)
dallas_2023 = read_in_dallas_data(2023)

In [39]:
dallas_combined = pd.concat([dallas_2014, dallas_2015, dallas_2016, dallas_2017, dallas_2018,
                             dallas_2019, dallas_2020, dallas_2021, dallas_2022, dallas_2023])

In [40]:
# resorting combined dataframe by outcome date
dallas_combined.sort_values(by = 'outcome_date', ascending = False, inplace = True)

# dropping duplicates by animal_id to focus on most recent observation per unique animal
print(dallas_combined.shape)
dallas_combined.drop_duplicates(subset = 'animal_id', inplace = True)
print(dallas_combined.shape)
dallas_combined.head()

(344923, 34)
(275158, 34)


,animal_id,animal_type,animal_breed,kennel_number,kennel_status,tag_type,activity_number,activity_sequence,source_id,census_tract,council_district,intake_type,intake_subtype,intake_total,reason,staff_id,intake_date,intake_time,due_out,intake_condition,hold_request,outcome_type,outcome_date,outcome_time,receipt_number,impound_number,service_request_number,outcome_condition,chip_status,animal_origin,additional_information,month,year,outcome_subtype
42903,A1229376,CAT,DOMESTIC SH,FREEZER,UNAVAILABLE,NaN,NaN,1,P1110761,3902.0,7.0,DISPOS REQ,OTC,1.0,OTHRINTAKS,JVW,2024-10-04,17:06:00,10/04/2024,DECEASED,ADOP RESCU,DISPOSAL,2027-10-04,18:00:00,NaN,K24-644567,NaN,DECEASED,SCAN NO CHIP,OVER THE COUNTER,NaN,FY2024,FY2024,DISPOSAL
43357,A1229851,DOG,MIXED BREED,B17,AVAILABLE,NaN,NaN,1,P1111387,5200.0,1.0,STRAY,AT LARGE,1.0,OTHRINTAKS,JVW,2024-10-09,17:42:00,10/15/2024,APP WNL,ADOP RESCU,ADOPTION,2024-10-27,17:50:00,R24-622569,K24-645135,NaN,APP WNL,SCAN CHIP,OVER THE COUNTER,NaN,FY2024,FY2024,WESTMORELD
39489,A1225816,CAT,DOMESTIC SH,413,UNAVAILABLE,NaN,NaN,1,P1101968,NaN,NaN,FOSTER,APPOINT,1.0,SURGERY,BEC,2024-10-26,10:35:00,10/26/2024,APP WNL,ADOP RESCU,ADOPTION,2024-10-27,14:15:00,R24-622542,K24-646788,NaN,APP WNL,SCAN CHIP,OVER THE COUNTER,NaN,FY2024,FY2024,WESTMORELD
16153,A1204135,DOG,MIXED BREED,FOSTER,PENDING,NaN,NaN,1,P1083409,NaN,NaN,FOSTER,APPOINT,1.0,FOR ADOPT,JLC,2024-10-27,12:15:00,10/27/2024,APP WNL,ADOP RESCU,ADOPTION,2024-10-27,12:15:00,R24-622533,K24-646916,NaN,APP WNL,SCAN CHIP,OVER THE COUNTER,NaN,FY2024,FY2024,BY FOSTER
44442,A1231147,DOG,CHIHUAHUA SH,K03,AVAILABLE,NaN,NaN,1,P1113395,12900.0,9.0,OWNER SURRENDER,WALK IN,1.0,PERSNLISSU,GRA,2024-10-24,18:14:00,10/24/2024,APP WNL,ADOP RESCU,ADOPTION,2024-10-27,14:53:00,R24-622444,K24-646666,NaN,APP WNL,SCAN NO CHIP,OVER THE COUNTER,BBW,FY2024,FY2024,WESTMORELD


In [41]:
dallas_combined.info()

<class 'pandas.core.frame.DataFrame'>
Index: 275158 entries, 42903 to 44634
Data columns (total 34 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   animal_id               275158 non-null  object        
 1   animal_type             275157 non-null  object        
 2   animal_breed            275042 non-null  object        
 3   kennel_number           275158 non-null  object        
 4   kennel_status           275158 non-null  object        
 5   tag_type                2 non-null       object        
 6   activity_number         114170 non-null  object        
 7   activity_sequence       275158 non-null  int64         
 8   source_id               275158 non-null  object        
 9   census_tract            228718 non-null  object        
 10  council_district        228718 non-null  object        
 11  intake_type             275158 non-null  object        
 12  intake_subtype          270831 n

*tag_type and service_request_number are primarily null values, going to drop these columns. Also going to drop several columns that don't have an impact on an animal's outcome type.*

In [42]:
dallas_combined.drop(columns = ['kennel_number', 'kennel_status', 'tag_type', 'activity_number',
                                'activity_sequence', 'source_id', 'census_tract', 'council_district',
                                'intake_subtype', 'intake_total', 'staff_id', 'intake_time', 'due_out',
                                'hold_request', 'outcome_time', 'receipt_number', 'impound_number',
                                'service_request_number', 'additional_information', 'month', 'year',
                                'outcome_subtype'], inplace = True)

In [43]:
# see animal types
dallas_combined['animal_type'].value_counts()

animal_type
DOG          191178
CAT           65699
WILDLIFE       9724
BIRD           8275
LIVESTOCK       280
D                 1
Name: count, dtype: int64

*Same as the data from Austin shelters, we are going to ignore anything that isn't a cat or dog.*

In [44]:
# dropping all animal types that aren't cats or dogs
print(dallas_combined.shape)
dallas_combined = dallas_combined[dallas_combined['animal_type'] != 'WILDLIFE']
dallas_combined = dallas_combined[dallas_combined['animal_type'] != 'BIRD']
dallas_combined = dallas_combined[dallas_combined['animal_type'] != 'LIVESTOCK']
dallas_combined = dallas_combined[dallas_combined['animal_type'] != 'D']
print(dallas_combined.shape)

(275158, 12)
(256878, 12)


In [45]:
# see remainging null values
dallas_combined.isnull().sum()

animal_id                 0
animal_type               1
animal_breed              1
intake_type               0
reason               121159
intake_date               0
intake_condition          0
outcome_type              0
outcome_date           1136
outcome_condition     22135
chip_status           18127
animal_origin         18210
dtype: int64

*Going to create a duration column for how long an animal is in the shelter. Dropping observations with no outcome date first.*

In [46]:
# drop null outcome date rows
dallas_combined.dropna(subset = 'outcome_date', inplace = True)

# column for stay duration
dallas_combined['stay_duration'] = dallas_combined['outcome_date'] - dallas_combined['intake_date']
# convert to number of days
dallas_combined['stay_duration'] = dallas_combined['stay_duration'].dt.days.astype(int)

# check for negative stay_duration
dallas_combined[dallas_combined['stay_duration'] < 0]['stay_duration'].value_counts()

stay_duration
-333    3
-30     2
-332    2
-330    1
-331    1
-300    1
-55     1
-698    1
-272    1
Name: count, dtype: int64

*Just a few instances with negative stays, just dropping these rows.*

In [47]:
dallas_combined = dallas_combined[dallas_combined['stay_duration'] >= 0]

*Looking at the columns with remaing null values.*

In [48]:
dallas_combined['reason'].unique()

array(['OTHRINTAKS', 'SURGERY', 'FOR ADOPT', 'PERSNLISSU', 'OTHER',
       'NOTRIGHTFT', nan, 'SHORT-TERM', 'TRANSFER', 'BEHAVIOR', 'MEDICAL',
       'HOUSING', 'FINANCIAL', 'EVICTION', 'TNR CLINIC', 'STRAY',
       'TOO MANY', 'OWNER PROBLEM', 'AGGRESSIVE - PEOPLE',
       'AGGRESSIVE - ANIMAL', 'HOUSE SOIL', 'UNKNOWN',
       'DESTRUCTIVE AT HOME', 'OTHER PET', 'ESCAPES', 'CAUTIONCAT',
       'CHILD PROBLEM', 'INJURED', 'ILL', 'MOVE', 'LANDLORD', 'VOCAL',
       'COST', 'ALLERGIC', 'NO TIME', 'BITES', 'HYPER',
       'KILLED ANOTHER ANIMAL', 'NEW BABY', 'TOO BIG', 'MOVE APT',
       'DEAD ON ARRIVAL', 'OWNER DIED', 'TRAVEL', 'ATTENTION',
       'DESTRUCTIVE OUTSIDE', 'FOUND ANIM', 'RESPONSIBLE', 'TOO OLD',
       'EUTHANASIA OLD', 'ABANDON', 'AFRAID', 'FOSTER', 'BLIND/DEAF',
       'CRUELTY', 'NOFRIENDLY', 'NO HOME', 'DEAF', 'EUTHANASIA ILL',
       'NO YARD', 'BLIND', 'CHASES PEOPLE', 'DISOBIDIEN', 'FENCE',
       'QUARANTINE', 'EUTHANASIA BEHAV', 'WRONG SEX', 'JUMPS UP',
       'RE

In [49]:
dallas_combined['outcome_condition'].unique()

array(['DECEASED', 'APP WNL', nan, 'APP SICK', 'CRITICAL', 'APP INJ',
       'UNDERAGE', 'GERIATRIC', 'FATAL', 'UNKNOWN', 'DEAD',
       'TREATABLE REHABILITABLE NON-CONTAGIOUS',
       'UNHEALTHY UNTREATABLE NON-CONTAGIOUS', 'HEALTHY',
       'TREATABLE MANAGEABLE NON-CONTAGIOUS',
       'UNHEALTHY UNTREATABLE CONTAGIOUS',
       'TREATABLE REHABILITABLE CONTAGIOUS',
       'TREATABLE MANAGEABLE CONTAGIOUS'], dtype=object)

In [50]:
dallas_combined['chip_status'].value_counts()

chip_status
SCAN NO CHIP                 163049
SCAN CHIP                     58427
UNABLE TO SCAN                16502
WILDLIFE - UNABLE TO SCAN         4
WILDLIFE - UNABEL TO SCAN         1
Name: count, dtype: int64

In [51]:
dallas_combined['animal_origin'].unique()

array(['OVER THE COUNTER', 'AGGOPS', 'COM CAT', nan, 'FIELD', 'BITE',
       'HART', 'PSPICKUP', 'OPS', 'AGGDD', 'CARE', 'SWEEP', 'RAPID',
       'NIGHT DROP'], dtype=object)

* reason: These could be pertinent to an animal being adopted or not, will fill nulls with 'none' that no reason was given.
* outcome_condition: We are going to assume if nothing was entered the animals was healthy.
* chip_status: We are assuming this will relate somewhat to outcome type, that if they are able to scan a chip they will get returned to owner. Dropping this column.
* animal_origin: Not even sure what all of these entries are, dropping this column.
* animal_breed: Didn't investigate, just dropping these null rows

In [52]:
dallas_combined.isnull().sum()

animal_id                 0
animal_type               1
animal_breed              1
intake_type               0
reason               120649
intake_date               0
intake_condition          0
outcome_type              0
outcome_date              0
outcome_condition     21070
chip_status           17746
animal_origin         17829
stay_duration             0
dtype: int64

In [53]:
# drop chip_status and animal_origin columns
dallas_combined = dallas_combined.drop(columns=['chip_status', 'animal_origin'])

# fill nulls for reason and outcome_condition
dallas_combined = dallas_combined.fillna({'reason': 'NONE', 'outcome_condition': 'HEALTHY'})

# drop null breed observations
dallas_combined.dropna(subset = 'animal_breed', inplace = True)

In [54]:
dallas_combined.info()

<class 'pandas.core.frame.DataFrame'>
Index: 255728 entries, 42903 to 1146
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   animal_id          255728 non-null  object        
 1   animal_type        255728 non-null  object        
 2   animal_breed       255728 non-null  object        
 3   intake_type        255728 non-null  object        
 4   reason             255728 non-null  object        
 5   intake_date        255728 non-null  datetime64[ns]
 6   intake_condition   255728 non-null  object        
 7   outcome_type       255728 non-null  object        
 8   outcome_date       255728 non-null  datetime64[ns]
 9   outcome_condition  255728 non-null  object        
 10  stay_duration      255728 non-null  int32         
dtypes: datetime64[ns](2), int32(1), object(8)
memory usage: 22.4+ MB


In [55]:
# saving combined data
dallas_combined.to_csv('../data/dallas-combined-shelter-data.csv', index=False)

---
## Data Dictionary

All data used for this project is from [City of Austin Open Data](https://data.austintexas.gov/) and [City of Dallas Open Data](https://www.dallasopendata.com/). The Austin shelter data is specifically from [Austin Animal Center Intakes](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Intakes/wter-evkm/about_data) and [Austin Animal Center Outcomes](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Outcomes/9t4d-g238/about_data). The Dallas shelter data is from [Dallas Animal Shelter Data Fiscal Year 2014-2015](https://www.dallasopendata.com/Archive/Dallas-Animal-Shelter-Data-Fiscal-Year-2014-2015/4j5h-8vay/about_data), [2015-2016](https://www.dallasopendata.com/Archive/Dallas-Animal-Shelter-Data-Fiscal-Year-2015-2016/bg5d-mj5u/about_data), [2016-2017](https://www.dallasopendata.com/Archive/Dallas-Animal-Shelter-Data-Fiscal-Year-2016-2017/sjyj-ydcj/about_data), [2017-2018](https://www.dallasopendata.com/Services/Dallas-Animal-Shelter-Data-Fiscal-Year-2017-2018/wb7n-sdxi/about_data), [2018-2019](https://www.dallasopendata.com/Services/Dallas-Animal-Shelter-Data-Fiscal-Year-2018-2019/kf5k-aswg/about_data), [2019-2020](https://www.dallasopendata.com/Services/Dallas-Animal-Shelter-Data-Fiscal-Year-2019-2020/7h2m-3um5/about_data), [2020-2021](https://www.dallasopendata.com/Services/Dallas-Animal-Shelter-Data-Fiscal-Year-2020-2021/sq59-vp2t/about_data), [2021-2022](https://www.dallasopendata.com/Services/Dallas-Animal-Shelter-Data-Fiscal-Year-2021-2022/uu3b-ppfz/about_data), [2022-2023](https://www.dallasopendata.com/Services/Dallas-Animal-Shelter-Data-Fiscal-Year-2022-2023/f77p-sgrc/about_data), and [2023-2025](https://www.dallasopendata.com/Services/Dallas-Animal-Shelter-Data-Fiscal-Year-2023-2025/uyte-zi7f/about_data)

|feature|type|description|source|
|---|---|---|---|
|**animal_id**|*str*|Unique animal ID|Austin & Dallas|
|**outcome_time**|*datetime*|Day and time of animal outcome|Austin|
|**date_of_birth**|*datetime*|Animal's date of birth|Austin|
|**outcome_type**|*str*|Outcome of animal|Austin & Dallas|
|**outcome_gender**|*str*|Neuter/spay status at outcome|Austin|
|**outcome_age**|*float*|Animal age at outcome|Austin|
|**intake_time**|*datetime*|Day and time animal is taken in by shelter|Austin|
|**found_location**|*str*|Where animal is found|Austin|
|**intake_type**|*str*|How animal is taken in|Austin & Dallas|
|**intake_condition**|*str*|Animal's health condition upon intake|Austin & Dallas|
|**animal_type**|*str*|Type of animal|Austin & Dallas|
|**intake_gender**|*str*|Neuter/spay status upon intake|Austin|
|**intake_age**|*float*|Animal age in months upon intake|Austin|
|**breed**|*str*|Animal breed|Austin & Dallas|
|**color**|*str*|Animal color|Austin|
|**stay_duration**|*str*|How many days animal is in shelter before outcome|Austin & Dallas|
|**spay_neuter**|*int*|1 if an animal is spayed or neutered while in the shelter|Austin|
|**reason**|*str*|Why an animal is taken in by a shelter|Dallas|
|**outcome_condition**|*str*|Animal's health condition upon outcome|Dallas|

In future notebooks we will explore the relationships these features have on animal's shelter outcomes.